# Playing with data

read data from csv

In [1]:
import pandas as pd
df_season = pd.read_csv("static/data/NBA_season.csv")
df_season = df_season[df_season.columns.tolist()[1:]]
df_playoffs = pd.read_csv("static/data/NBA_playoffs.csv")
df_playoffs= df_playoffs[df_playoffs.columns.tolist()[1:]]
df_teams = pd.read_csv("static/data/NBA_teams.csv")

In [2]:
df_teams

,Tm,Win%
0,BOS,58.9
1,DEN,50.7
2,VAN,42.5
3,LAL,59.4
4,POR,53.6
...,...,...
99,SDS,38.4
100,ROC,45.5
101,CAP,45.2
102,MMP,41.7


add team win rate to df_season

In [3]:
def get_win_rate(flag):
    try:
        win_rate = df_teams[df_teams['Tm'] == flag].iloc[0]['Win%']
    except KeyError:
        # team has no record of win rate
        win_rate = None
    return win_rate

df_season['TmWin%'] = df_season['Tm'].apply(get_win_rate)

In [4]:
df_season

,id,Name,From,To,Height,Weight,Year,Age,Tm,Lg,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TmWin%,score2
0,abdelal01,Alaa Abdelnaby,1991,1995,208.28,108.862169,1991,22.0,POR,NBA,...,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,53.6,2.455228
1,abdelal01,Alaa Abdelnaby,1991,1995,208.28,108.862169,1992,23.0,POR,NBA,...,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,53.6,4.646871
2,abdulma02,Mahmoud Abdul-Rauf,1991,2001,185.42,73.481964,1991,21.0,DEN,NBA,...,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,50.7,9.554548
3,abdulma02,Mahmoud Abdul-Rauf,1991,2001,185.42,73.481964,1992,22.0,DEN,NBA,...,1.1,1.4,2.4,0.5,0.0,1.4,1.6,10.3,50.7,7.143728
4,abdulma02,Mahmoud Abdul-Rauf,1991,2001,185.42,73.481964,1993,23.0,DEN,NBA,...,2.1,2.8,4.2,1.0,0.1,2.3,2.2,19.2,50.7,14.192043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8289,zellety01,Tyler Zeller,2013,2020,213.36,114.758870,2016,26.0,BOS,NBA,...,1.9,3.0,0.5,0.2,0.4,0.8,1.6,6.1,58.9,4.532796
8290,zellety01,Tyler Zeller,2013,2020,213.36,114.758870,2017,27.0,BOS,NBA,...,1.6,2.4,0.8,0.1,0.4,0.4,1.2,3.5,58.9,3.098721
8291,zipsepa01,Paul Zipser,2017,2018,203.20,97.522360,2017,22.0,CHI,NBA,...,2.5,2.8,0.8,0.3,0.4,0.9,1.8,5.5,51.1,3.887579
8292,zipsepa01,Paul Zipser,2017,2018,203.20,97.522360,2018,23.0,CHI,NBA,...,2.2,2.4,0.9,0.4,0.3,0.8,1.6,4.0,51.1,2.976722


filter players by these filters:

Playoffs:
---------
Games >= 3
year >= 1978

Season:
-------
Games >= 40
Team win rate >= 48%
year >= 1978

In [5]:
df_season = df_season[(df_season['G'] >= 40) & (df_season['Year'] >= 1978) & (df_season['TmWin%'] >= 48)]
df_playoffs = df_playoffs[(df_playoffs['G'] >= 3) & (df_playoffs['Year'] >= 1978)]

fill 0 where None

In [6]:
df_season = df_season.fillna(0)
df_playoffs = df_playoffs.fillna(0)

add score column according to the calculation

In [7]:
from utils import calc_score
df_season['score2'] = df_season.apply(calc_score, axis=1)
df_playoffs['score2'] = df_playoffs.apply(calc_score, axis=1)

save as csvs

In [8]:
df_season.to_csv("NBA_season.csv")
df_playoffs.to_csv("NBA_playoffs.csv")
df_teams.to_csv("NBA_teams.csv")

## Calculate average stats by pos

In [9]:
group = df_season[(df_season['Year'] >= 1980) & (df_season['MP'] >= 20)].groupby('Pos')

df_avg_stats_by_pos_mixed = group[["AST", "PTS", "TOV", "BLK", "STL", "ORB", "DRB"]].mean()
df_avg_stats_by_pos = df_avg_stats_by_pos_mixed.loc[['PF', 'C', 'PG', 'SF', 'SG']]
df_count_by_pos = group['id'].count()

In [10]:
count_by_pos = {
    'C': df_count_by_pos.loc['C'],
    'PF': df_count_by_pos.loc['PF'],
    'PG': df_count_by_pos.loc['PG'],
    'SF': df_count_by_pos.loc['SF'],
    'SG': df_count_by_pos.loc['SG'],
}

for pos, row in df_avg_stats_by_pos_mixed.iterrows():
    if pos not in ['PF', 'C', 'PG', 'SF', 'SG']:
        # get amount of players with this mixed position
        amount_mixed = df_count_by_pos.loc[pos]

        # get the sum of the mixed stats for each stat
        stats = []
        for stat in row:
            stats.append(stat * amount_mixed)

        # for each position, add avg to position
        for p in pos.split(','):
            p = p.strip()
            amount_pure = count_by_pos[p]

            # get average stats for this pure pos
            pos_row = df_avg_stats_by_pos.loc[p]
            for i in range(len(stats)):
                # create new weighted arithmetic mean
                new_stat = (pos_row[i] * amount_pure + stats[i]) / (amount_pure + amount_mixed)
                pos_row[i] = new_stat

            # update amount for this pos
            count_by_pos[p] += amount_mixed

In [11]:
df_avg_stats_by_pos

,AST,PTS,TOV,BLK,STL,ORB,DRB
Pos,,,,,,,
PF,1.949051,13.090512,1.768501,0.792694,0.848292,2.149146,5.082638
C,1.637459,12.275679,1.771553,1.434745,0.707383,2.469707,5.542997
PG,5.697905,12.883524,2.184571,0.198571,1.284857,0.674095,2.477524
SF,2.516463,14.219003,1.799059,0.504610,1.034807,1.398589,3.579210
SG,3.156130,14.305517,1.815937,0.308581,1.099650,0.859370,2.641856


## Calculate player score for season and playoffs

option 1 - best year of the player

In [12]:
df_season[['id', 'Name', 'Year', 'score2']].groupby('id').agg({'Name': 'first', 'Year': 'first', 'score2': 'max'}).sort_values(by='score2', ascending=False)

,Name,Year,score2
id,,,
nealll01,Lloyd Neal,1978,33.071961
jamesaa01,Aaron James,1978,30.759931
jordami01,Michael Jordan,1985,29.800270
westbru01,Russell Westbrook,2009,28.530249
hardeja01,James Harden,2010,28.425531
...,...,...,...
nilesmi01,Mike Niles,1981,1.487358
kornefr01,Frank Kornet,1990,1.471596
lewisra01,Ralph Lewis,1988,1.327903


In [13]:
df_playoffs[['id', 'Name', 'Year', 'score2']].groupby('id').agg({'Name': 'first', 'Year': 'first', 'score2': 'max'}).sort_values(by='score2', ascending=False)

,Name,Year,score2
id,,,
smithel01,Elmore Smith,1978,48.363019
nealll01,Lloyd Neal,1978,33.071961
jamesaa01,Aaron James,1978,30.759931
jordami01,Michael Jordan,1985,29.800270
westbru01,Russell Westbrook,2009,28.530249
...,...,...,...
kiddst01,Stanton Kidd,2020,0.342254
kutluib01,Ibo Kutluay,2005,0.212021
mccrach01,Chris McCray,2007,0.151961


option 2 - average of last max 4 years

In [14]:
rows_season = {}
rows_playoffs = {}

for player in df_season['id'].unique():
    df_player_season = df_season[df_season['id'] == player].sort_values(by='score2', ascending=False).head(4)
    df_player_playoffs = df_season[df_season['id'] == player].sort_values(by='score2', ascending=False).head(4)
    avg_season = df_player_season['score2'].mean()
    avg_playoffs = df_player_playoffs['score2'].mean()

    rows_season[player] = [df_player_season.iloc[0]['Name'], list(df_player_season['Year'].sort_values()), avg_season]
    rows_playoffs[player] = [df_player_playoffs.iloc[0]['Name'], list(df_player_playoffs['Year'].sort_values()), avg_playoffs]

df_best4_season = pd.DataFrame().from_dict(rows_season, orient='index', columns=['Name', 'Years', 'score2'])
df_best4_playoffs = pd.DataFrame().from_dict(rows_playoffs, orient='index', columns=['Name', 'Years', 'score2'])

In [15]:
df_best4_season.sort_values(by='score2', ascending=False)

,Name,Years,score2
nealll01,Lloyd Neal,[1978],33.071961
jordami01,Michael Jordan,"[1987, 1988, 1989, 1990]",28.843035
perrycu01,Curtis Perry,[1978],28.318568
hardeja01,James Harden,"[2017, 2018, 2019, 2020]",27.494824
antetgi01,Giannis Antetokounmpo,"[2019, 2020, 2021, 2022]",25.823249
...,...,...,...
kornefr01,Frank Kornet,[1990],1.471596
hayesst01,Steve Hayes,"[1984, 1986]",1.470879
lewisra01,Ralph Lewis,[1988],1.327903
blankla01,Lance Blanks,[1992],1.168212


In [16]:
df_best4_playoffs.sort_values(by='score2', ascending=False)

,Name,Years,score2
nealll01,Lloyd Neal,[1978],33.071961
jordami01,Michael Jordan,"[1987, 1988, 1989, 1990]",28.843035
perrycu01,Curtis Perry,[1978],28.318568
hardeja01,James Harden,"[2017, 2018, 2019, 2020]",27.494824
antetgi01,Giannis Antetokounmpo,"[2019, 2020, 2021, 2022]",25.823249
...,...,...,...
kornefr01,Frank Kornet,[1990],1.471596
hayesst01,Steve Hayes,"[1984, 1986]",1.470879
lewisra01,Ralph Lewis,[1988],1.327903
blankla01,Lance Blanks,[1992],1.168212
